In [ ]:
!pip install tensorflow==1.15 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 50 kB 8.0 MB/s 
     |████████████████████████████████| 3.8 MB 42.8 MB/s 
     |████████████████████████████████| 503 kB 67.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=b6b5fdeb27443197f2cf30cab988e325db90911e4076fcd3a9d6a1c3b20d1436
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting un

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
%cd /gdrive/MyDrive/

/gdrive/MyDrive


In [ ]:
import sys
#sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from mytraining import train_network
import tensorflow as tf

# Generate data

In [ ]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

# Set up model and training parameters

this are the parameters used in the paper to get the best model

In [ ]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0    #lambda2
params['loss_weight_sindy_x'] = 1e-4  #lambda1
params['loss_weight_sindy_regularization'] = 1e-5 #lambdad3

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 3001
params['refinement_epochs'] = 501

# Run training experiments

In [ ]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0
   training loss 0.04036667197942734, (0.039020013, 316.71616, 13.378942, 0.8763885)
   validation loss 0.041519809514284134, (0.04044957, 367.64114, 10.614735, 0.8763885)
decoder loss ratio: 0.220330, decoder SINDy loss  ratio: 1.009713
Epoch 100
   training loss 0.00011111702042398974, (6.935376e-05, 0.50983256, 0.23397717, 1.8365543)
   validation loss 0.0001172778065665625, (7.530801e-05, 0.69370884, 0.23604257, 1.8365543)
decoder loss ratio: 0.000410, decoder SINDy loss  ratio: 0.022453
Epoch 200
   training loss 4.00125136366114e-05, (1.6327635e-05, 0.09158667, 0.048899405, 1.8794938)
   validation loss 4.121959864278324e-05, (1.7317063e-05, 0.19145222, 0.051075976, 1.8794938)
decoder loss ratio: 0.000094, decoder SINDy loss  ratio: 0.004859
Epoch 300
   training loss 2.658869379956741e-05, (7.4607005e-06, 0.044927854, 0.021713927, 1.6956601)
   validation loss 2.8523008950287476e-05, (8.733858e-06, 0.114567965, 0.028325498, 1.6956601)
decoder loss r